# Embedding Texts 

## Importing Libraries

In [5]:
!pip install transformers torch tqdm

In [16]:
!pip install --upgrade torch torchvision torchaudio


In [1]:
import torch
from transformers import CLIPTokenizer, CLIPTextModel
import numpy as np
from tqdm import tqdm

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
CAPTIONS_FILE = '/Users/harshakhiyani/My_Stuff/CODE/GITHUB/Visual-Situmlai-Reconstruction-Using-fMRI-and-Deep-Learning/NSD-Flat Data Proccessing/prepared_nsd_data_subj01/captions.txt'
SAVE_PATH = '/Users/harshakhiyani/My_Stuff/CODE/GITHUB/Visual-Situmlai-Reconstruction-Using-fMRI-and-Deep-Learning/NSD-Flat Data Proccessing/Caption Embedding/Caption Embdedding.npy'

In [3]:
with open(CAPTIONS_FILE,'r') as f:
    captions = [line.strip() for line in f.readlines()]

This takes out all the lines of the captions and removes the new line character

In [4]:
tokenizer = CLIPTokenizer.from_pretrained('openai/clip-vit-base-patch32')
model = CLIPTextModel.from_pretrained("openai/clip-vit-base-patch32",use_safetensors=True)
model.eval()              

CLIPTextModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 512)
      (position_embedding): Embedding(77, 512)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=512, out_features=2048, bias=True)
            (fc2): Linear(in_features=2048, out_features=512, bias=True)
          )
          (layer_norm2): LayerNorm((512,), eps=1e

ClLIPTokenizer turns the words to numbers that the computer can understand.

CLIPTextModel turns the numbers to meaningful vectors that have simantic meaning

In [5]:
device = torch.device("mps" if torch.cuda.is_available() else "cpu")
model.to(device)


CLIPTextModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 512)
      (position_embedding): Embedding(77, 512)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=512, out_features=2048, bias=True)
            (fc2): Linear(in_features=2048, out_features=512, bias=True)
          )
          (layer_norm2): LayerNorm((512,), eps=1e

In [6]:
features = []

In [7]:
print("CLIP text embeddings for ", len(captions)," captions")

CLIP text embeddings for  27892  captions


In [8]:
# Read and Learn this function
for captions in tqdm(captions):
    tokens = tokenizer(captions,return_tensors='pt',padding = "max_length",truncation=True,max_length=15)
    tokens = {k:v.to(device) for k, v in tokens.items()}
    with torch.no_grad():
        outputs = model(**tokens)
        last_hidden_state = outputs.last_hidden_state
        embedding = last_hidden_state.view(-1).cpu().numpy()
        features.append(embedding)


100%|█████████████████████████████████████| 27892/27892 [16:37<00:00, 27.96it/s]


tokenizer converts caption into token IDs numbers for model
this returns a Pytorch Tensor 
All captions will have 15 tokens if smaller they will get padding, if bigger they will get truncated

As we are not predicting anything we turn grads to off 
model function runs text encoder and gives one embedding vector of 768 dimensions per token

In [9]:
features = np.stack(features)
np.save(SAVE_PATH, features)
print(f"✅ Done! Saved embeddings with shape: {features.shape}")


✅ Done! Saved embeddings with shape: (27892, 7680)
